In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local[*]').appName("Classificação com Spark").getOrCreate()

In [4]:
spark

Carregamento dos Dados

In [5]:
dados = spark.read.csv('/content/dados_clientes.csv', sep=',', header=True, inferSchema=True)

In [6]:
dados

DataFrame[id: int, Churn: string, Mais65anos: int, Conjuge: string, Dependentes: string, MesesDeContrato: int, TelefoneFixo: string, MaisDeUmaLinhaTelefonica: string, Internet: string, SegurancaOnline: string, BackupOnline: string, SeguroDispositivo: string, SuporteTecnico: string, TVaCabo: string, StreamingFilmes: string, TipoContrato: string, ContaCorreio: string, MetodoPagamento: string, MesesCobrados: double]

In [7]:
dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|               Nao|               Sim|              

In [8]:
dados.count()

10348

In [9]:
dados.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [10]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



Transformando os dados

In [11]:
colunasBinarias = [
                   'Churn',
                   'Conjuge',
                   'Dependentes',
                   'TelefoneFixo',
                   'MaisDeUmaLinhaTelefonica',
                   'SegurancaOnline',
                   'BackupOnline',
                   'SeguroDispositivo',
                   'SuporteTecnico',
                   'TVaCabo',
                   'StreamingFilmes',
                   'ContaCorreio'
]

In [12]:
from pyspark.sql import functions as f

In [14]:
todasColunas = [f.when(f.col(c)=='Sim', 1).otherwise(0).alias(c) for c in colunasBinarias]

In [16]:
for coluna in reversed(dados.columns):
  if coluna not in colunasBinarias:
    todasColunas.insert(0, coluna)
todasColunas


['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN =(Churn, 'Sim') THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN =(Conjuge, 'Sim') THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN =(Dependentes, 'Sim') THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN =(TelefoneFixo, 'Sim') THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN =(MaisDeUmaLinhaTelefonica, 'Sim') THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN =(SegurancaOnline, 'Sim') THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN =(BackupOnline, 'Sim') THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN =(SeguroDispositivo, 'Sim') THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN =(SuporteTecnico, 'Sim') THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN =(TVaCabo, 'Sim') THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN =(StreamingFilmes, 'Sim') THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column

In [17]:
dados.select(todasColunas).show()

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

In [18]:
dataset = dados.select(todasColunas)

In [19]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)



Criando Dummies

In [20]:
dados.select(['Internet','TipoContrato','MetodoPagamento']).show()

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|   CartaoCredito|
|        DSL| Mensalmente|          Boleto|
|FibraOptica| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|   DebitoEmConta|
|        DSL| Mensalmente|          Boleto|
|        Nao|    DoisAnos|   CartaoCredito|
|FibraOptica|       UmAno|   CartaoCredito|
|FibraOptica| Mensalmente|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica|    DoisAnos|   CartaoCredito|
|        Nao|       UmAno|          Boleto|
|FibraOptica|    DoisAnos|   DebitoEmConta|
|        DSL| Mensalmente|   CartaoCredito|
|FibraOptica| Mensalmente|Boleto

In [21]:
dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0).show()

+----+---+-----------+---+
|  id|DSL|FibraOptica|Nao|
+----+---+-----------+---+
|7982|  1|          0|  0|
|9465|  0|          1|  0|
|2122|  1|          0|  0|
|3997|  1|          0|  0|
|6654|  0|          1|  0|
|7880|  0|          1|  0|
|4519|  0|          1|  0|
|6466|  0|          1|  0|
| 496|  1|          0|  0|
|7833|  0|          1|  0|
|1591|  0|          0|  1|
|2866|  0|          1|  0|
|8592|  0|          1|  0|
|1829|  0|          1|  0|
| 463|  0|          1|  0|
|4900|  0|          1|  0|
|4818|  0|          1|  0|
|7554|  1|          0|  0|
|1342|  0|          0|  1|
|5300|  0|          1|  0|
+----+---+-----------+---+
only showing top 20 rows


In [22]:
Internet = dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0)
TipoContrato = dataset.groupBy('id').pivot('TipoContrato').agg(f.lit(1)).na.fill(0)
MetodoPagamento = dataset.groupBy('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0)

In [23]:
TipoContrato.show()

+----+--------+-----------+-----+
|  id|DoisAnos|Mensalmente|UmAno|
+----+--------+-----------+-----+
|7993|       0|          1|    0|
|8592|       0|          1|    0|
|4519|       0|          0|    1|
|1088|       0|          1|    0|
|1238|       0|          1|    0|
|1342|       1|          0|    0|
|4935|       0|          0|    1|
| 471|       0|          1|    0|
|5518|       0|          1|    0|
| 463|       0|          1|    0|
|3794|       0|          1|    0|
|9465|       0|          1|    0|
|7240|       0|          1|    0|
|9852|       0|          1|    0|
|1959|       0|          1|    0|
|7754|       0|          1|    0|
|5156|       0|          0|    1|
|6658|       0|          1|    0|
|6397|       0|          0|    1|
|1829|       0|          1|    0|
+----+--------+-----------+-----+
only showing top 20 rows


In [24]:
dataset\
  .join(Internet, 'id','inner')\
  .join(TipoContrato, 'id','inner')\
  .join(MetodoPagamento, 'id','inner')\
  .select(
      '*',
      f.col('DSL').alias('Internet_DSL'),
      f.col('FibraOptica').alias('Internet_FibraOptica'),
      f.col('Nao').alias('Internet_Nao'),
      f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
      f.col('UmAno').alias('TipoContrato_UmAno'),
      f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
      f.col('DebitoEmConta').alias('MetodoPatamento_DebitoEmConta'),
      f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
      f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
      f.col('Boleto').alias('MetodoPagamento_Boleto')
  )\
  .drop(
      'Internet','TipoContrato','MetodoPagamento','DSL','FibraOptica','Nao',
      'Mensalmente','UmAno','DoisAnos','DebitoEmConta','CartaoCredito',
      'BoletoEletronico','Boleto'
  )\
  .show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPatamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [25]:
dataset = dataset\
  .join(Internet, 'id','inner')\
  .join(TipoContrato, 'id','inner')\
  .join(MetodoPagamento, 'id','inner')\
  .select(
      '*',
      f.col('DSL').alias('Internet_DSL'),
      f.col('FibraOptica').alias('Internet_FibraOptica'),
      f.col('Nao').alias('Internet_Nao'),
      f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
      f.col('UmAno').alias('TipoContrato_UmAno'),
      f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
      f.col('DebitoEmConta').alias('MetodoPatamento_DebitoEmConta'),
      f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
      f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
      f.col('Boleto').alias('MetodoPagamento_Boleto')
  )\
  .drop(
      'Internet','TipoContrato','MetodoPagamento','DSL','FibraOptica','Nao',
      'Mensalmente','UmAno','DoisAnos','DebitoEmConta','CartaoCredito',
      'BoletoEletronico','Boleto'
  )

In [26]:
dataset.show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPatamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [27]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContr